In [44]:
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio
file_name = """229r/task_date_{'x': 10, 'y': 10}_{'NOT': 1.0, 'NAND': 1.0, 'AND': 2.0, 'ORN': 2.0, 'OR': 3.0, 'ANDN': 3.0, 'NOR': 4.0, 'XOR': 4.0, 'EQU': 5.0}_03-06_20-38-48.txt"""

def plot_graph(file_name="""229r/task_date_{'x': 10, 'y': 10}_{'NOT': 1.0, 'NAND': 1.0, 'AND': 2.0, 'ORN': 2.0, 'OR': 3.0, 'ANDN': 3.0, 'NOR': 4.0, 'XOR': 4.0, 'EQU': 5.0}_03-06_20-38-48.txt""", title='Average Fitness over Time', width = 800,    height = 600):

    # Read data from file into a list of strings
    with open(file_name, 'r') as f:
        lines = f.readlines()

    # Parse the lines to create a list of dictionaries, where each dictionary contains values for each column
    data = []
    for line in lines:
        if line.startswith('UD:'):
            values = line.strip().split()
            # print(values)

            data.append({
                'UD': int(values[1]),
                'Gen': float(values[3]),
                'Fit': float(values[5]),
                'Orgs': int(values[7])
            })

    # Create dataframe from list of dictionaries
    df = pd.DataFrame(data)
        

    # Define x and y data
    x_data = df.iloc[:, 0] # First column as x-axis
    y_data = df[["Fit"]]
    # y_data = df.iloc[:, 1:] # Every other column as y-axis

    # Create plotly trace for each column
    data = []
    for column in y_data.columns:
        trace = go.Scatter(x=x_data, y=y_data[column], mode='lines', name=column)
        data.append(trace)

    # Define plot layout
    layout = go.Layout(title=title, xaxis=dict(title='# of updates'), yaxis=dict(title='average fitness'), margin=dict(t=40, b=20, l=20, r=20), autosize=False, width=width, height=height)

    # Create plotly figure
    fig = go.Figure(data=data, layout=layout)

    # Display plot
    file_parts = file_name.split('/')
    fig.show()
    pio.write_image(fig, file_parts[0]+'/plots/'+file_parts[1].split('.')[0]+'.png', width=width, height=height, scale=2)


In [45]:
import os
import pandas as pd
import re

# Directory containing the files
dir_path = '229r/all1 copy'

# Directory containing the files
dat_dir_path = 'data/229r/all1 copy'

# List to store the file names and last lines
file_data = []

# Regular expression pattern
pattern = r"(\d+)_mxc_(\w+)_(\[.*\])_date_(\d{2}-\d{2})_(\d{2}-\d{2}-\d{2})_xydict_values\((\[.*\])\)\.txt"
# Create an empty DataFrame with the required columns
column_names = ['index', 'max_count', 'values', 'experiment_start_time_string', 'xy_values', 'Last_UD', 'Gen', 'Fit', 'Orgs']
df = pd.DataFrame(columns=column_names)

# Create an empty list to store the data
data_list = []

# Loop through all the files in the directory
for filename in os.listdir(dir_path):
    # Check if the file is a text file
    if filename.endswith('.txt'):
        # Open the file and read the last line
        with open(os.path.join(dir_path, filename), 'r') as f:
            # Search for pattern in the log_file_name
            match = re.search(pattern, filename)
            if not match:
                raise Exception("No match found for pattern in file name: {}".format(filename))
            index = int(match.group(1))
            max_count = match.group(2) == 'True'
            values = [int(i) for i in match.group(3).strip('[]').split(', ')]
            experiment_start_time_string = match.group(4) + "_" + match.group(5)
            xy_values = [int(i) for i in match.group(6).strip('[]').split(', ')]
            
            lines = f.readlines()
            last_line = lines[-1].strip()
            
            with open(os.path.join(dat_dir_path, filename), 'r') as fdat: 
                lines_dat = fdat.readlines()
                last_line_dat = lines_dat[-1].strip()
                numbers = last_line_dat.split()

                dictionary = {
                    "Update": int(numbers[0]),
                    "Not": int(numbers[1]),
                    "Nand": int(numbers[2]),
                    "And": int(numbers[3]),
                    "OrNot": int(numbers[4]),
                    "Or": int(numbers[5]),
                    "AndNot": int(numbers[6]),
                    "Nor": int(numbers[7]),
                    "Xor": int(numbers[8]),
                    "Equals": int(numbers[9])
                }

            
            if last_line.startswith('UD:'):
                values = last_line.strip().split()

                data = {
                'index': index,
                'max_count': max_count,
                'values': values,
                'experiment_start_time_string': experiment_start_time_string,
                'xy_values': xy_values,
                'Last_UD': int(values[1]),
                'Gen': float(values[3]),
                'Fit': float(values[5]),
                'Orgs': int(values[7])
                }
                data.update(dictionary)

                # Append the data to the list
                data_list.append(data)
            else: 
                raise Exception("Last line does not start with UD: {}".format(filename))

# Create the DataFrame from the data list
df = pd.DataFrame(data_list)

# Print the dataframe
df


,index,max_count,values,experiment_start_time_string,xy_values,Last_UD,Gen,Fit,Orgs,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor,Equals
0,0,True,"[UD:, 4400, Gen:, 1017.648, Fit:, 5.304728e+72...",04-13_20-35-15,"[120, 120]",4400,1017.6480,5.304728e+72,14345,4400,12397,13294,11875,13104,49,542,9074,0,1826
1,0,False,"[UD:, 10000, Gen:, 1331.495, Fit:, 1.073964e+1...",04-13_15-10-40,"[120, 120]",10000,1331.4950,1.073964e+158,14259,10000,11353,404,12,10494,11233,121,2247,0,0
2,1,True,"[UD:, 10000, Gen:, 2259.621, Fit:, 16367.07, O...",04-14_03-56-46,"[120, 120]",10000,2259.6210,1.636707e+04,14389,10000,13162,13658,6,12150,12124,13590,12012,0,0
3,1,True,"[UD:, 10000, Gen:, 1634.431, Fit:, 9830.949, O...",04-14_03-18-04,"[120, 120]",10000,1634.4310,9.830949e+03,14329,10000,12849,10478,10673,12931,11769,5349,11334,0,0
4,0,True,"[UD:, 5000, Gen:, 1007.769, Fit:, 1.389793e+21...",04-13_22-02-37,"[120, 120]",5000,1007.7690,1.389793e+211,14322,5000,13787,13143,8502,13129,7929,4635,7501,3736,1186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,0,False,"[UD:, 1800, Gen:, 256.2159, Fit:, 2.363671e+19...",04-13_16-42-23,"[120, 120]",1800,256.2159,2.363671e+197,14346,1800,3628,2254,1401,4012,61,1727,945,0,0
125,1,True,"[UD:, 2800, Gen:, 1088.078, Fit:, 6.218935e+09...",04-14_07-34-42,"[120, 120]",2800,1088.0780,6.218935e+09,14350,2800,13094,13374,5543,13004,6732,2792,1055,0,5964
126,0,False,"[UD:, 1200, Gen:, 160.7237, Fit:, 5.020909e+22...",04-13_18-43-54,"[120, 120]",1200,160.7237,5.020909e+229,13903,1200,3097,149,0,3547,88,0,0,0,0
127,2,True,"[UD:, 10000, Gen:, 1649.042, Fit:, 3653.363, O...",04-14_09-01-14,"[120, 120]",10000,1649.0420,3.653363e+03,14363,10000,13137,13592,0,13360,12541,11406,12850,0,0


In [46]:
import pandas as pd
import pickle

# Pickle the dataframe to a file
with open('229r/results/all1_copy_fitness_and_counts_df.pickle', 'wb') as f:
    pickle.dump(df, f)


In [47]:
import pandas as pd
import pickle

# Load the pickled dataframe from the file
with open('229r/results/all1_copy_fitness_and_counts_df.pickle', 'rb') as f:
    df = pickle.load(f)

# Print the dataframe
display(df)


,index,max_count,values,experiment_start_time_string,xy_values,Last_UD,Gen,Fit,Orgs,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor,Equals
0,0,True,"[UD:, 4400, Gen:, 1017.648, Fit:, 5.304728e+72...",04-13_20-35-15,"[120, 120]",4400,1017.6480,5.304728e+72,14345,4400,12397,13294,11875,13104,49,542,9074,0,1826
1,0,False,"[UD:, 10000, Gen:, 1331.495, Fit:, 1.073964e+1...",04-13_15-10-40,"[120, 120]",10000,1331.4950,1.073964e+158,14259,10000,11353,404,12,10494,11233,121,2247,0,0
2,1,True,"[UD:, 10000, Gen:, 2259.621, Fit:, 16367.07, O...",04-14_03-56-46,"[120, 120]",10000,2259.6210,1.636707e+04,14389,10000,13162,13658,6,12150,12124,13590,12012,0,0
3,1,True,"[UD:, 10000, Gen:, 1634.431, Fit:, 9830.949, O...",04-14_03-18-04,"[120, 120]",10000,1634.4310,9.830949e+03,14329,10000,12849,10478,10673,12931,11769,5349,11334,0,0
4,0,True,"[UD:, 5000, Gen:, 1007.769, Fit:, 1.389793e+21...",04-13_22-02-37,"[120, 120]",5000,1007.7690,1.389793e+211,14322,5000,13787,13143,8502,13129,7929,4635,7501,3736,1186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,0,False,"[UD:, 1800, Gen:, 256.2159, Fit:, 2.363671e+19...",04-13_16-42-23,"[120, 120]",1800,256.2159,2.363671e+197,14346,1800,3628,2254,1401,4012,61,1727,945,0,0
125,1,True,"[UD:, 2800, Gen:, 1088.078, Fit:, 6.218935e+09...",04-14_07-34-42,"[120, 120]",2800,1088.0780,6.218935e+09,14350,2800,13094,13374,5543,13004,6732,2792,1055,0,5964
126,0,False,"[UD:, 1200, Gen:, 160.7237, Fit:, 5.020909e+22...",04-13_18-43-54,"[120, 120]",1200,160.7237,5.020909e+229,13903,1200,3097,149,0,3547,88,0,0,0,0
127,2,True,"[UD:, 10000, Gen:, 1649.042, Fit:, 3653.363, O...",04-14_09-01-14,"[120, 120]",10000,1649.0420,3.653363e+03,14363,10000,13137,13592,0,13360,12541,11406,12850,0,0


In [48]:
df.describe()

,index,Last_UD,Gen,Fit,Orgs,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor,Equals
count,129.000000,129.000000,129.000000,1.290000e+02,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000
mean,0.651163,6595.007752,1601.899445,3.414343e+298,12931.968992,6595.348837,8618.705426,8035.356589,3441.046512,8203.093023,5499.713178,5435.511628,3894.620155,1030.395349,1829.007752
std,0.657305,3810.532608,1234.001866,inf,2462.867162,3810.467911,4965.531273,5324.024210,4934.044157,5167.173377,5478.534349,5622.560514,5062.408634,2884.615928,3593.211026
min,0.000000,700.000000,57.988240,4.338221e-01,4732.000000,700.000000,119.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2000.000000,502.390000,6.450240e+04,12824.000000,2000.000000,3628.000000,2063.000000,8.000000,3093.000000,29.000000,15.000000,0.000000,0.000000,0.000000
50%,1.000000,9100.000000,1634.431000,1.103311e+34,14224.000000,9100.000000,10953.000000,9781.000000,112.000000,11243.000000,2993.000000,2480.000000,170.000000,0.000000,0.000000
75%,1.000000,10000.000000,2253.908000,5.582985e+139,14353.000000,10000.000000,13110.000000,13254.000000,8859.000000,12785.000000,11777.000000,11724.000000,9552.000000,2.000000,1186.000000
max,2.000000,10000.000000,7114.001000,3.391337e+300,14391.000000,10000.000000,13989.000000,13824.000000,13296.000000,14077.000000,13722.000000,13855.000000,12857.000000,11911.000000,11855.000000


In [49]:
df.columns

Index(['index', 'max_count', 'values', 'experiment_start_time_string',
       'xy_values', 'Last_UD', 'Gen', 'Fit', 'Orgs', 'Update', 'Not', 'Nand',
       'And', 'OrNot', 'Or', 'AndNot', 'Nor', 'Xor', 'Equals'],
      dtype='object')

In [50]:
# Get all the rows with max_count = False and Last_UD = 10000
filtered_df = df[(df['max_count'] == False) & (df['Last_UD'] == 10000)]

In [51]:
filtered_df

,index,max_count,values,experiment_start_time_string,xy_values,Last_UD,Gen,Fit,Orgs,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor,Equals
1,0,False,"[UD:, 10000, Gen:, 1331.495, Fit:, 1.073964e+1...",04-13_15-10-40,"[120, 120]",10000,1331.4950,1.073964e+158,14259,10000,11353,404,12,10494,11233,121,2247,0,0
14,1,False,"[UD:, 10000, Gen:, 706.1867, Fit:, 5.579337e+1...",04-14_06-14-08,"[120, 120]",10000,706.1867,5.579337e+119,12930,10000,4741,140,0,1956,3589,10,21,0,0
30,2,False,"[UD:, 10000, Gen:, 4381.775, Fit:, 3.019758e+2...",04-14_08-28-00,"[120, 120]",10000,4381.7750,3.019758e+282,14046,10000,981,187,13,1861,12644,1458,11342,0,0
46,1,False,"[UD:, 10000, Gen:, 2458.093, Fit:, 7.271464e+4...",04-14_02-37-03,"[120, 120]",10000,2458.0930,7.271464e+42,14063,10000,3967,667,714,13846,13722,12852,4573,2,2
62,1,False,"[UD:, 10000, Gen:, 7114.001, Fit:, 2.021980e+2...",04-14_00-38-55,"[120, 120]",10000,7114.0010,2.021980e+271,14224,10000,310,12969,419,11798,249,13855,0,9117,1
72,0,False,"[UD:, 10000, Gen:, 1819.951, Fit:, 7.043458e+1...",04-13_19-45-47,"[120, 120]",10000,1819.9510,7.043458e+116,14019,10000,13692,13740,21,427,4,1,0,0,0
81,2,False,"[UD:, 10000, Gen:, 2350.410, Fit:, 1.932565e+4...",04-14_09-42-13,"[120, 120]",10000,2350.4100,1.932565e+40,14015,10000,8270,457,0,7455,6737,0,26,0,0
82,1,False,"[UD:, 10000, Gen:, 1698.857, Fit:, 1.508958e+1...",04-14_01-08-45,"[120, 120]",10000,1698.8570,1.508958e+135,13958,10000,942,248,21,12271,5105,12179,6332,0,0
83,0,False,"[UD:, 10000, Gen:, 440.5190, Fit:, 2.603604e+7...",04-13_16-59-10,"[120, 120]",10000,440.5190,2.603604e+75,4732,10000,551,89,14,790,11,545,0,0,0
87,1,False,"[UD:, 10000, Gen:, 3164.719, Fit:, 7.995238e+9...",04-14_04-26-24,"[120, 120]",10000,3164.7190,7.995238e+90,13463,10000,6544,4739,290,12701,91,54,24,0,1
